In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import cvxpy as cp
import networkx as nx
import matplotlib.pyplot as plt

from time import time
from itertools import product
from matplotlib.patches import Rectangle

In [ ]:
np.random.seed(1)
grid_points = np.arange(101)
box_sides = [2, .5]
boxes = []
for i in grid_points:
    for j in grid_points:
        center = np.array([i, j])
        np.random.shuffle(box_sides)
        sides = np.diag(box_sides).dot(np.random.rand(2))
        boxes.append((center - sides, center + sides))

In [ ]:
def plot_boxes(boxes):
    for i, box in enumerate(boxes):
#         plt.text(*(box[1] + box[0])/2, i)
        plt.gca().add_patch(Rectangle(box[0], *(box[1] - box[0]), fc='lightcyan', ec='k'))
    plt.xlim([-2, len(grid_points) + 2])
    plt.ylim([-2, len(grid_points) + 2])

plt.figure(figsize=(9, 9))
plot_boxes(boxes)

start = time()
pairs = np.array([(i, j) for i in range(len(boxes)) for j in range(i)])
min_pairs = np.array([(b1[0], b2[0]) for i, b1 in enumerate(boxes) for b2 in boxes[:i]])
max_pairs = np.array([(b1[1], b2[1]) for i, b1 in enumerate(boxes) for b2 in boxes[:i]])
a = np.max(min_pairs, axis=1)
b = np.min(max_pairs, axis=1)
d = b - a
idx = [i for i, di in enumerate(d) if all(di >= 0)]
time() - start, len(pairs[idx])

bottom = np.array([b[0] for b in boxes])
top = np.array([b[1] for b in boxes])
limits = np.hstack((bottom, top))
start = time()
vertices = []
for i, l1 in enumerate(zip(bottom, top)):
    for j, l2 in enumerate(zip(bottom[:i], top[:i])):
        if all(np.maximum(l1[0], l2[0]) <= np.minimum(l1[1], l2[1])):
            vertices.append((i, j))
time() - start, len(vertices)

In [ ]:
s = 's'
t = 't'
vertices = [s, t]
inter = {i: [] for i in range(len(boxes))}
intersections = {}
centers = {}
start = time()
for i, box1 in enumerate(boxes):
    for j, box2 in enumerate(boxes[:i]):
        bl = np.maximum(box1[0], box2[0])
        tr = np.minimum(box1[1], box2[1])
        if all(bl <= tr):
            v = (i, j)
            vertices.append(v)
            intersections[v] = (bl, tr)
            centers[v] = (bl + tr) / 2
            inter[i].append(v)
            inter[j].append(v)
time() - start

In [ ]:
start = time()
edges = []    
lengths = []
for i in inter:
    for u, v in product(inter[i], inter[i]):
        if u != v:
            edges.append((u, v))
            duv = np.linalg.norm(centers[u] - centers[v])
            lengths.append(duv)
time() - start

In [ ]:
out_s = [(s, v) for v in vertices[2:] if v[1] == 0]
inc_t = [(v, t) for v in vertices[2:] if v[0] == len(boxes) - 1]
aux_edges = out_s + inc_t
edges += aux_edges
lengths += [0] * len(aux_edges)

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(vertices)
G.add_edges_from(edges)
for e, l in zip(edges, lengths):
    G.edges[e]['weight'] = l

In [ ]:
tic = time()
path = nx.shortest_path(G, source=s, target=t, weight='weight')
time() - tic

In [ ]:
import cvxpy as cp

variables = {}
constraints = []
cost = 0
x_prev = None
for v in path[1:-1]:
    variables[v] = cp.Variable(2)
    constraints.append(variables[v] >= intersections[v][0])
    constraints.append(variables[v] <= intersections[v][1])
    if x_prev is not None:
        cost += cp.norm(variables[v] - x_prev, 2)
    x_prev = variables[v]

prob = cp.Problem(cp.Minimize(cost), constraints)
prob.solve()
traj_opt = np.array([x.value for x in variables.values()])
prob.solver_stats.solve_time

In [ ]:
traj = np.array([centers[v] for v in path[1:-1]])
length = sum(np.linalg.norm(y - x) for x, y in zip(traj[:-1], traj[1:]))

plt.figure(figsize=(9, 9))
plot_boxes(boxes)
plt.plot(*traj.T, c='r')
plt.plot(*traj_opt.T, c='b')
plt.savefig('path_planning.pdf', bbox_inches='tight')
length, prob.value